In [ ]:
%matplotlib widget
from bmcs_shear.shear_crack.beam_design import RCBeamDesign
from bmcs_shear.api import CrackBridgeAdv
from bmcs_shear.shear_crack.crack_propagation import CrackPropagation
#from bmcs_shear.shear_crack.crack_extension import CrackExtension
import numpy as np
np.seterr(divide='ignore', invalid='ignore') ;

In [ ]:
ce = CrackPropagation(n_seg=25)
ce.sz_cp.trait_set(x_00=1250, n_m=12)
ce.sz_bd.trait_set(H=400, L=2000, B=50);
ce.sz_bd.csl.add_layer(CrackBridgeAdv(z=9, n=2, d_s=10, E = 210000, tau = 16, sig_y = 713, 
                   dowel_factor = 1, f_c = 33.3)
)
ce.sz_cp.sz_ctr.ell=15
ce.sz_ctss.L_cs = 100
ce.sz_cp.add_x_tip_an([1250,100])

In [ ]:
ce.sz_ctss.sig_x_tip_0

In [ ]:
ce.sz_ctss.sig_z_tip_1

In [ ]:
ce.sz_ctss.psi_k

In [ ]:
ce.sz_ctss.tau_x_tip_1k

In [ ]:
import sympy as sp
sp.init_printing()

In [ ]:
tau_fps, sigma_x, sigma_y = sp.symbols(r'tau, sigma_x, sigma_y')
sigma_1, sigma_2 = sp.symbols(r'sigma_1, sigma_2')
f_ct, f_cm = sp.symbols(r'f_ct, f_cm', nonnegative=True)

sigma_xy = sp.Matrix([[sigma_x, tau_fps],
                     [tau_fps, sigma_y]])
sigma_12 = sp.Matrix([[sigma_1, 0],
                      [0, sigma_2]])

P_xy, D_xy = sigma_xy.diagonalize()

sigma_1_xy = D_xy[0,0]
sigma_2_xy = D_xy[1,1]

Kupfer_ct = sp.Eq(sigma_2 / f_ct - sp.Rational(8,10) * sigma_1 / f_cm, 1)

sigma_2_ct_solved = sp.solve(Kupfer_ct, sigma_2)[0]

sig_2_ct_eq = sp.Eq(sigma_2_ct_solved, sigma_2_xy)

sig_2_ct_eq_xy = sig_2_ct_eq.subs(sigma_1, sigma_1_xy)

tau_fps_ct_solved = sp.solve(sig_2_ct_eq_xy, tau_fps)[0]

get_tau_fps = sp.lambdify((sigma_x, sigma_y, f_cm, f_ct), tau_fps_ct_solved, 'numpy')

psi = sp.atan( sp.simplify(-P_xy[0,0] / P_xy[1,0])).subs(tau_fps, tau_fps_ct_solved)

get_psi = sp.lambdify((sigma_x, sigma_y, f_cm, f_ct), psi, 'numpy')

In [ ]:
f_ct = ce.sz_ctss.f_t
#        print('f_ct', f_ct)
f_cm = ce.sz_ctss.f_c
#        print('f_cm', f_cm)
sigma_x = ce.sz_ctss.sig_x_tip_0
#        print('sigma_x', sigma_x)
sigma_y = ce.sz_ctss.sig_z_tip_1
#        print('sigma_y', sigma_y)
tau_x_tip_1k = get_tau_fps(sigma_x, sigma_y, f_cm, f_ct)
#        print('tau_x_tip_1k', tau_x_tip_1k)


In [ ]:
sigma_x, sigma_y, tau_x_tip_1k